## Population Grrowth

In [1]:
import pandas as pd
import os
current_dir = os.getcwd()

In [3]:
url_population_growth = os.path.join(current_dir, '../../data/raw/extractions/Densidad poblacion_y _Crecimiento_poblacional.xlsx')

df = pd.read_excel(url_population_growth, engine='openpyxl'  )

# Mostrar las primeras filas
print(df.head())

  table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)  \
0  Table 1. Annual Estimates of the Resident Popu...                                                             
1                                    Geographic Area                                                             
2                                                NaN                                                             
3                                      United States                                                             
4                                          Northeast                                                             

            Unnamed: 1      Unnamed: 2                          Unnamed: 3  \
0                  NaN             NaN                                 NaN   
1  2010-04-01 00:00:00             NaN  Population Estimate (as of July 1)   
2               Census  Estimates Base                                2010   
3  

In [4]:
# Seleccionar columnas relevantes
columnas_deseadas = [
    'table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)',
    'Unnamed: 4',  # 2011
    'Unnamed: 5',  # 2012
    'Unnamed: 6',  # 2013
    'Unnamed: 7',  # 2014
    'Unnamed: 8',  # 2015
    'Unnamed: 9',  # 2016
    'Unnamed: 10'  # 2017
]

df_filtrado = df[columnas_deseadas]

# Filtrar filas que no sean estados
df_filtrado = df_filtrado[~df_filtrado[columnas_deseadas[0]].str.contains('United States|Puerto Rico', na=False)]
df_final = df_filtrado.iloc[8:].reset_index(drop=True)

# Renombrar columnas para que tengan sentido
df_final = df_final.rename(columns={
    columnas_deseadas[0]: 'State',
    columnas_deseadas[1]: 'Poblacion_2011',
    columnas_deseadas[2]: 'Poblacion_2012',
    columnas_deseadas[3]: 'Poblacion_2013',
    columnas_deseadas[4]: 'Poblacion_2014',
    columnas_deseadas[5]: 'Poblacion_2015',
    columnas_deseadas[6]: 'Poblacion_2016',
    columnas_deseadas[7]: 'Poblacion_2017',
})


# Quitar puntos al inicio del nombre de los estados
df_final['State'] = df_final['State'].str.lstrip('.')

# Calcular crecimiento interanual
for año in range(2011, 2017):
    col_inicio = f'Poblacion_{año}'
    col_fin = f'Poblacion_{año + 1}'
    col_crecimiento = f'Crecimiento_p_{año}'
    df_final[col_crecimiento] = ((df_final[col_fin] - df_final[col_inicio]) / df_final[col_inicio] * 100).round(2)

# Seleccionar columnas finales: Estado + crecimiento por año
columnas_resultado = ['State'] + [f'Crecimiento_p_{año}' for año in range(2011, 2017)]
df_crecimiento = df_final[columnas_resultado]

# Mostrar resultado
df_crecimiento.head()


,State,Crecimiento_p_2011,Crecimiento_p_2012,Crecimiento_p_2013,Crecimiento_p_2014,Crecimiento_p_2015,Crecimiento_p_2016
0,Arizona,1.27,1.19,1.47,1.47,1.63,1.48
1,Arkansas,0.39,0.25,0.27,0.36,0.40,0.38
2,California,0.82,0.82,0.88,0.83,0.64,0.49
3,Colorado,1.40,1.47,1.54,1.88,1.63,1.31
4,Connecticut,0.17,0.01,-0.01,-0.21,-0.25,-0.14


In [5]:
estados_deseados = ['California', 'Texas', 'Wisconsin']
df_crecimiento_filtrado = df_crecimiento[df_crecimiento['State'].isin(estados_deseados)].reset_index(drop=True)
df_crecimiento_filtrado

,State,Crecimiento_p_2011,Crecimiento_p_2012,Crecimiento_p_2013,Crecimiento_p_2014,Crecimiento_p_2015,Crecimiento_p_2016
0,California,0.82,0.82,0.88,0.83,0.64,0.49
1,Texas,1.71,1.52,1.83,1.88,1.62,1.36
2,Wisconsin,0.26,0.29,0.26,0.16,0.20,0.30


In [ ]:
# Diccionario de estados a abreviaturas
abreviaturas = {
    'California': 'CA',
    'Texas': 'TX',
    'Wisconsin': 'WI'
}

# Reemplazar los nombres por abreviaturas
df_crecimiento_filtrado['State'] = df_crecimiento_filtrado['State'].replace(abreviaturas)
df_crecimiento_filtrado

,State,Crecimiento_p_2011,Crecimiento_p_2012,Crecimiento_p_2013,Crecimiento_p_2014,Crecimiento_p_2015,Crecimiento_p_2016
0,CA,0.82,0.82,0.88,0.83,0.64,0.49
1,TX,1.71,1.52,1.83,1.88,1.62,1.36
2,WI,0.26,0.29,0.26,0.16,0.20,0.30


In [7]:
## Guardar en CSV

df_crecimiento_filtrado_path = os.path.join(current_dir, '../../data/raw/macrodata')

df_crecimiento_poblacion_data = os.path.join(df_crecimiento_filtrado_path, "population_growth.csv")

df_crecimiento_filtrado.to_csv(df_crecimiento_poblacion_data, index=False)